In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Data Loading (Same) ---
file_path = "../1900_2021_DISASTERS.xlsx - emdat data.csv"
try:
    df_raw = pd.read_csv(file_path)
    print(f"Raw data loaded. Total rows: {len(df_raw)}")
except FileNotFoundError:
    print(f"ERROR: '{file_path}' not found. Please check the path.")
    # raise

# --- 2. Time Series Engineering (Same) ---
continent_filter = "Asia"
disaster_type_filter = "Flood"

print(f"Filtering: Continent='{continent_filter}', Type='{disaster_type_filter}'")

df_filtered = df_raw[
    (df_raw['Continent'] == continent_filter) &
    (df_raw['Disaster Type'] == disaster_type_filter)
].copy()

df_filtered['Start Day'] = 1
df_filtered = df_filtered.dropna(subset=['Start Year', 'Start Month'])
df_filtered['datetime'] = pd.to_datetime(
    df_filtered[['Start Year', 'Start Month', 'Start Day']].rename(
        columns={'Start Year': 'year', 'Start Month': 'month', 'Start Day': 'day'}
    )
)
print(f"Date conversion complete. Found {len(df_filtered)} '{disaster_type_filter}' events.")

# --- 3. Data Resampling (CORRECTED) ---

df_ts = df_filtered.set_index('datetime')

# b. 'resample('ME')' -> Resample the data into MONTHLY (Month-End) periods (Now 'ME' instead of 'M')
#    '.size()' -> Count the number of events falling into each month.
#    '.fillna(0)' -> If there were NO floods in a month (NaN),
#                   write 0 (zero) there. (Now 'fillna(0)' instead of 'asfreq(0)')
# --- CORRECTION: 'M' -> 'ME' and 'asfreq(0)' -> 'fillna(0)' ---
ts_df = df_ts.resample('ME').size().fillna(0)
# --- END CORRECTION ---

# c. Format the DataFrame into the form the 'Prophet' library likes
ts_df = ts_df.reset_index()
ts_df.columns = ['ds', 'y'] # 'ds' = Datestamp, 'y' = Target

print("\n--- Time Series (First 5 Rows) ---")
print(ts_df.head())
print("\n--- Time Series (Last 5 Rows) ---")
print(ts_df.tail())

# --- 4. Visualization (Same) ---
plt.figure(figsize=(16, 8))
plt.plot(ts_df['ds'], ts_df['y'])
plt.title(f"Time Series: '{disaster_type_filter}' Events in '{continent_filter}' Continent (Monthly)")
plt.xlabel("Date")
plt.ylabel("Monthly Event Count")
plt.show()